# Introduction 

This jupyter notebook contains a demo version of the SiamMask tracker. 
This demo is based on Qiang Wang's (@foolwood on github) implemantation which you can find at this adress : https://github.com/foolwood/SiamMask

It is designed to work in Google Colab 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zak-souid/DLAV_project/main/m2/Milestone2demo2.ipynb)


When trying to implement it with our milestone 1 we had issues with python finding the different libraries and did not manage to make it work in time. The issue still puzzles us.

#### Initial imports

In [19]:
from google.colab import drive
import os
drive.mount('/content/drive')
!pip install torch torchvision got10k opencv-python
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import torch
import pandas as pd
import os
import sys
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import namedtuple
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from got10k.trackers import Tracker
import glob
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js

# load the files for milestone 2
#if not os.path.exists('DLAV_project/siamfc-pytorch/'):
#  !git clone --branch milestone2 https://github.com/zak-souid/DLAV_project.git ;

import sys
# load the files for milestone 1

#obj_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
#hand_model = torch.hub.load('ultralytics/yolov5', 'custom', path='DLAV_project/best.pt')

import glob
from google.colab.patches import cv2_imshow
from os.path import exists, join, basename, splitext


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
git_repo_url = 'https://github.com/foolwood/SiamMask.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q --depth 1 {git_repo_url}
  !sed -i "/torch/d" {project_name}/requirements.txt
  !cd {project_name} && pip install -q -r requirements.txt
  !cd {project_name} && bash make.sh
  
import sys
sys.path.append(project_name)
sys.path.append(join(project_name, 'experiments', 'siammask_sharp'))
import time
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False

torch.set_grad_enabled(False)

from types import SimpleNamespace
from custom import Custom
from tools.test import *

In [21]:
exp_path = join(project_name, 'experiments/siammask_sharp')
pretrained_path1 = join(exp_path, 'SiamMask_DAVIS.pth')
pretrained_path2 = join(exp_path, 'SiamMask_VOT.pth')

# download pretrained weights
if not exists(pretrained_path1):
  !cd {exp_path} && wget http://www.robots.ox.ac.uk/~qwang/SiamMask_DAVIS.pth
if not exists(pretrained_path2):
  !cd {exp_path} && wget http://www.robots.ox.ac.uk/~qwang/SiamMask_VOT.pth
    
# init SiamMask
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg = load_config(SimpleNamespace(config=join(exp_path, 'config_davis.json')))
siammask = Custom(anchors=cfg['anchors'])
siammask = load_pretrain(siammask, pretrained_path1)
siammask = siammask.eval().to(device)

[2022-06-07 19:10:14,972-rk0-features.py# 66] Current training 0 layers:
	
[2022-06-07 19:10:14,976-rk0-features.py# 66] Current training 1 layers:
	
[2022-06-07 19:10:15,039-rk0-load_helper.py# 31] load pretrained model from SiamMask/experiments/siammask_sharp/SiamMask_DAVIS.pth
[2022-06-07 19:10:15,222-rk0-load_helper.py# 25] remove prefix 'module.'
[2022-06-07 19:10:15,227-rk0-load_helper.py# 18] used keys:356


#### Helper functions for the camera in colab

In [22]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
        js_reply: JavaScript object containing image from webcam
  Returns:
        img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
        bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [23]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[1].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment" }});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

#### Functions for Milestone 1

In [24]:
# Some functions used in next cell

def obj_detect(img):
    """
    Params:
          img: OpenCV BGR image
    Returns:
          res: pandas dataframe containing all objects detected by the model
    """
    results = obj_model(img)
    res = results.pandas().xyxy[0]  # img1 predictions (pandas)
    return res

def get_persons(objects):
    """
    Params:
          objects: pandas dataframe containing different objects
    Returns:
          persons: pandas dataframe containing only the objects labeled as person
    """
    persons = objects.loc[objects['name'] == "person"]
    idx_del = []
    areas = []

    for idx, pers in persons.iterrows():
        dx = pers["xmax"]-pers["xmin"]
        dy = pers["ymax"]-pers["ymin"]
        areas.append(dx*dy)
        if(areas[-1] < 10000):
            idx_del.append(idx)
    persons = persons.drop(idx_del)
    persons = persons.reset_index()


    return persons, areas

def get_personOfInterest(persons, hand, persons_area):  # returns None if rectangles don't intersect
    """
    Params:
          persons: pandas dataframe containing only objects labeled as person
          hand: pandas dataframe containing only the detected hands (labeled as 5)
    Returns:
          idx_max: integer corresponding to the index in the persons dataframe that corresponds to the person of interest
                   is equal to None if no hand is being detected or the hand does not belongs to any of the detectet persons
    """
    margin = 50 # For numerical inaccuracies
    idx_max = 0
    area_max = 0
    a_min = [-1,-1]
    a_max = [-1,-1]
    if not hand.empty:
        for idx, pers in persons.iterrows():
            dx = np.minimum(pers["xmax"], hand["xmax"].iloc[0]) - np.maximum(pers["xmin"], hand["xmin"].iloc[0])
            dy = np.minimum(pers["ymax"], hand["ymax"].iloc[0]) - np.maximum(pers["ymin"], hand["ymin"].iloc[0])
            if(dx<0 and dy<0):
                area = -dx*dy
            else:
                area = dx*dy
            if (area >= area_max - margin and persons_area[idx] >= persons_area[idx_max]):
                area_max = area
                idx_max = idx
                # actual overlapping area
                a_min = [int(np.maximum(pers["xmin"], hand["xmin"].iloc[0])),int(np.maximum(pers["ymin"], hand["ymin"].iloc[0]))]
                a_max = [int(np.minimum(pers["xmax"], hand["xmax"].iloc[0])),int(np.minimum(pers["ymax"], hand["ymax"].iloc[0]))]

    if (area_max>0):
        return idx_max, a_min, a_max
    else:
        return None, a_min, a_max

def hand_detect(img):
    """
    Params:
          img: OpenCV BGR image
    Returns:
          res: pandas dataframe containing all objects detected by the model
    """
    results = hand_model(img)
    res = results.pandas().xyxy[0]  # img1 predictions (pandas)
    return res

init = False
persOfInterest = False
bb_persOfInterest = [0,0,0,0]

def detect(img):
    global persOfInterest
    global bb_persOfInterest
    # detect all objects
    obj = obj_detect(img)
    # keep only the persons
    pers, persons_area = get_persons(obj)
    # detect hand
    hand = hand_detect(img)

    # identify person of interest
    idx_max, area_min, area_max = get_personOfInterest(pers,hand, persons_area)
    
    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    
    # draw bounding box of the hand (if multiple hands, just the first one)
    if (not hand.empty) and area_min!=None:
        bbox_array = cv2.rectangle(bbox_array,(int(hand["xmin"].iloc[0]),int(hand["ymin"].iloc[0])), (int(hand["xmax"].iloc[0]),int(hand["ymax"].iloc[0])), (0, 0, 255), 2)
        bbox_array = cv2.rectangle(bbox_array,tuple(area_min), tuple(area_max), (255, 0, 255), 2)

    # draw bounding boxes on overlay
    for index, row in pers.iterrows():
        start_point = (int(row["xmin"]), int(row["ymin"]))
        end_point = (int(row["xmax"]), int(row["ymax"]))
        if (index == idx_max):
            color = (0, 255, 0)
            persOfInterest = True
            bb_persOfInterest = [start_point[0], start_point[1], end_point[0]-start_point[0], end_point[1]-start_point[1]]
            #print(bb_persOfInterest)
        else:
            color = (255, 0, 0)
        thickness = 2
        bbox_array = cv2.rectangle(bbox_array,start_point, end_point, color, thickness)

    return bbox_array

# SIAMMASK

### Video feed

In [25]:
def start_demo():

  f = 0
# start streaming video from webcam
  video_stream()
# label for video
  label_html = 'Capturing...'
# initialze bounding box to empty
  bbox = ''
  count = 0 
  init_img = 0
  box_detect = [220,150,240,330]
  maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
  while (True):   
    js_reply = video_frame(label_html, bbox)

    frame = js_to_image(js_reply["img"])
    if f == 0:
      # detect a person on the first frame with Mask R-CNN
      maskrcnn = maskrcnn.eval()
      output = maskrcnn([torchvision.transforms.functional.to_tensor((frame[...,::-1] / 255).astype(np.float32))])[0]

      target_pos, target_sz = None, None
      for box, label, score in zip(output['boxes'], output['labels'], output['scores']):
        # search for a person with atleast 70% probability
        if score > 0.7 and label == 1:
          box = box.numpy()
          x, y = int(box[0]), int(box[1])
          w, h = int(box[2]) - x, int(box[3]) - y
          target_pos = np.array([x + w / 2, y + h / 2])
          target_sz = np.array([w, h])
          break
      assert target_pos is not None, "no person found on the first frame!"
      print("person found:", target_pos, target_sz)
      #target_pos, target_sz = bbox_persofinterest
      # init tracker
      state = siamese_init(frame, target_pos, target_sz, siammask, cfg['hp'], device=device)
      #state = detect(img)
    else:
      # track
      state = siamese_track(state, frame, mask_enable=True, refine_enable=True, device=device)
      location = state['ploygon'].flatten()
      mask = state['mask'] > state['p'].seg_thr
      
      frame[:, :, 2] = (mask > 0) * 255 + (mask == 0) * frame[:, :, 2]
      bbox_array = np.zeros([480,640,4], dtype=np.uint8)
      bbox_array = cv2.polylines(bbox_array, [np.int0(location).reshape((-1, 1, 2))], True, (255, 255, 0), 2)
      #cv2_imshow(frame)
      bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
      bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
      bbox = bbox_bytes
  
    f += 1
    # only on first 100 frames

# Demo

In [ ]:
start_demo()